# Dash Anomalías de temperatura

La apliclación desplegada tiene como objetivo de visualización los cambios de temperatura cercana a la superficie en todo el mundo, entre 1850 a 2018. El archivo presenta anomalías mensuales promedio de temperatura cercana a la superficie, en relación con el período 1961–1990.

En general, este conjunto de datos muestra un aumento del calentamiento promedio global desde mediados del siglo 19 y en los últimos años, en consonancia con otros análisis. Este fenómeno se debe a múltiples factores, incluidos una mejor representación del calentamiento del Ártico y una mejor comprensión de los sesgos en evolución en las mediciones sobre la superficie del mar y la tierra. Para revisar a detalle el proceso de obtención y procesamiento de los datos, visite la información de referencia al final de este documento.

El presente informe se encuentra estructurado por secciones, siguiendo el orden de construcción de la aplicación, explicando a detalle las herramientas utilizadas. Inicia con el procesamiento de la base de datos, continúa con el análisis exploratorio de datos y finaliza con el desarrollo de 3 modelos de pronósticos para la serie de Anomalías de Temperatura. Al final de este reporte se encuentra el código completo de la aplicación Dash.

In [20]:
# Importanción de librerías

import pandas as pd
from time import mktime
from datetime import datetime, timedelta
import time
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Temas para las figuras
sns.set_theme()
sns.set_context("paper")

## Manejo de la base de datos

* Previo a la construcción de la aplicación, es necesario procesar la base de datos. En este caso, utilizamos la librería pandas para leer los datos y configurar la estructura para que esté acorde con el formato de series de tiempo. 

In [4]:
# Leer los datos

df_temperature = pd.read_csv('./data/originaldata_temp.csv')
df_temperature.head()

,year,january,february,march,april,may,june,july,august,september,october,november,december,yearmean,Unnamed: 14
0,1850,-0.675,-0.333,-0.591,-0.589,-0.508,-0.344,-0.160,-0.208,-0.385,-0.533,-0.283,-0.404,-0.418,NaN
1,1850,53.000,47.000,44.000,48.000,49.000,49.000,52.000,55.000,57.000,55.000,58.000,58.000,NaN,NaN
2,1851,-0.201,-0.469,-0.646,-0.542,-0.198,-0.137,-0.097,-0.102,-0.091,-0.008,-0.082,-0.228,-0.233,NaN
3,1851,58.000,55.000,56.000,55.000,54.000,53.000,54.000,59.000,47.000,51.000,48.000,48.000,NaN,NaN
4,1852,-0.375,-0.477,-0.560,-0.585,-0.127,-0.084,0.005,-0.136,-0.002,-0.172,-0.305,0.065,-0.229,NaN


In [5]:
# Eliminar columna vacía

df_temperature = df_temperature.drop(df_temperature.columns[df_temperature.columns.str.contains('Unnamed')], axis=1)
df_temperature.head()

,year,january,february,march,april,may,june,july,august,september,october,november,december,yearmean
0,1850,-0.675,-0.333,-0.591,-0.589,-0.508,-0.344,-0.160,-0.208,-0.385,-0.533,-0.283,-0.404,-0.418
1,1850,53.000,47.000,44.000,48.000,49.000,49.000,52.000,55.000,57.000,55.000,58.000,58.000,NaN
2,1851,-0.201,-0.469,-0.646,-0.542,-0.198,-0.137,-0.097,-0.102,-0.091,-0.008,-0.082,-0.228,-0.233
3,1851,58.000,55.000,56.000,55.000,54.000,53.000,54.000,59.000,47.000,51.000,48.000,48.000,NaN
4,1852,-0.375,-0.477,-0.560,-0.585,-0.127,-0.084,0.005,-0.136,-0.002,-0.172,-0.305,0.065,-0.229


In [6]:
# Eliminar filas intermedias
df_temperature = df_temperature.drop(df_temperature.index[1::2]).reset_index(drop=True)
df_temperature.head()


,year,january,february,march,april,may,june,july,august,september,october,november,december,yearmean
0,1850,-0.675,-0.333,-0.591,-0.589,-0.508,-0.344,-0.160,-0.208,-0.385,-0.533,-0.283,-0.404,-0.418
1,1851,-0.201,-0.469,-0.646,-0.542,-0.198,-0.137,-0.097,-0.102,-0.091,-0.008,-0.082,-0.228,-0.233
2,1852,-0.375,-0.477,-0.560,-0.585,-0.127,-0.084,0.005,-0.136,-0.002,-0.172,-0.305,0.065,-0.229
3,1853,-0.233,-0.404,-0.280,-0.386,-0.268,-0.142,-0.083,-0.057,-0.250,-0.392,-0.411,-0.337,-0.270
4,1854,-0.381,-0.361,-0.243,-0.334,-0.291,-0.299,-0.179,-0.239,-0.217,-0.095,-0.410,-0.450,-0.292


* Aquí colocamos los meses que se encuentra en las filas como variables independientes en una sola columna, con su respectivo valor de anomalía, usando la función melt.

In [12]:
# Convertir a formato de serie de tiempo
df_timeseries = pd.melt(df_temperature, id_vars=['year'])

# Eliminar filas donde la columna 'variable' sea igual a 'yearmean'
df_timeseries = df_timeseries[df_timeseries['variable'] != 'yearmean']

# Cambiar el nombre de la columna 'variable' a 'month'
df_timeseries = df_timeseries.rename(columns={'variable':'month'})

df_timeseries.head()


,year,month,value
0,1850,january,-0.675
1,1851,january,-0.201
2,1852,january,-0.375
3,1853,january,-0.233
4,1854,january,-0.381


In [17]:
# Crear la nueva columna de fecha en formato %Y-%m
df_timeseries['date'] = pd.to_datetime(df_timeseries['year'].astype(str) + '-' + df_timeseries['month'], format='%Y-%B')

# Formatear la fecha para que solo muestre mes y año
df_timeseries['date'] = df_timeseries['date'].dt.strftime('%Y-%m')
df_timeseries


,year,month,value,date
0,1850,january,-0.675,1850-01
174,1850,february,-0.333,1850-02
348,1850,march,-0.591,1850-03
522,1850,april,-0.589,1850-04
696,1850,may,-0.508,1850-05
...,...,...,...,...
869,2023,may,0.871,2023-05
1043,2023,june,1.052,2023-06
1217,2023,july,1.150,2023-07
1391,2023,august,1.199,2023-08


In [16]:
df_timeseries = df_timeseries.sort_values(by=['date'])

# Eliminar las últimas tres filas
df_timeseries = df_timeseries.iloc[:-3]
df_timeseries

,year,month,value,date
0,1850,january,-0.675,1850-01
174,1850,february,-0.333,1850-02
348,1850,march,-0.591,1850-03
522,1850,april,-0.589,1850-04
696,1850,may,-0.508,1850-05
...,...,...,...,...
869,2023,may,0.871,2023-05
1043,2023,june,1.052,2023-06
1217,2023,july,1.150,2023-07
1391,2023,august,1.199,2023-08


In [18]:
df_timeseries.to_csv("df_timeseries.csv", index=False)

# Construcción del dash

* El dashboard está dividido en dos secciones principales: la sección de **navegación** a la izquierda y la sección de **contenido** a la derecha.

* **Sección de navegación:**
Esta sección, ubicada a la izquierda, contiene tres botones interactivos que permiten cambiar la visualización de la información en la sección de contenido. Los botones están diseñados con una tipografía clara y legible, como Arial o Helvetica, y están espaciados uniformemente para facilitar la navegación. Cada botón tiene un título descriptivo correspondiente a su función: "Incio", "Análisis exploratorio de datos (EDA)" y "Modelos predictivos".

* **Sección de contenido:**
Esta sección, ubicada a la derecha, muestra la información correspondiente al botón seleccionado en la sección de navegación.

    1. **Inicio:** Esta ventana proporciona una visión general del conjunto de datos, proporcionando su información de acceso.

    2. **Serie de tiempo (EDA):** Esta ventana presenta un análisis de los datos. Incluye gráficos de la serie de tiempo en diversas escalas temporales (mensual, anual, decenal), gráficos de línea, gráficos de caja y bigotes, mapas de calor, proporcionando información sobre patrones y tendencias de la variable objetivo (Anomalía de temperatura (°C)) de forma interactiva. Asimismo, se explora la estacionariedad de la serie, información base para la construcción de modelos predictivos.

    3. **Modelos predictivos:** Aquí presenta los resultados de tres modelos predictivos: ARIMA, suavización exponencial y regresión polinómica. Cada modelo se presenta en un gráfico separado que muestra los datos históricos y las predicciones del modelo. También se proporcionan medidas de precisión del modelo, como el error cuadrático medio (MSE) y el coeficiente de determinación (R^2).

* El primer paso para la construcción del dash fue la creación de la plantilla de trabajo separando el dash en dos secciones principales: navegación y contenido:

```Python
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)


app = dash.Dash(__name__, suppress_callback_exceptions=True)


app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Main(style={'display':'grid', 'height':'100%','minHeight':'100vh','gridTemplateColumns': '50vh 1fr', 'gridTemplateRows': '50px 1fr 30px', }, children=[
       
        html.Nav(className='nav', style={'display':'fixed','gridArea':'nav', 'gridColumnStart':'1','gridColumnEnd':'2','gridRowStart':'1','gridRowEnd':'5', 'backgroundColor': 'white','border':'1px solid #cbc4ad', 'borderRadius':'5px', 'padding': '20px'}, children=[
            html.H1('Anomalia de Temperatura de Superficie Global', style={'color': 'black', 'marginBottom': '20px', 'fontSize':'1.5rem', 'textAlign':'center'}),
            html.H2('¿Cuáles han sido las variaciones más significativas de la temperatura de la superficie global a lo largo del tiempo?', style={'color': 'black', 'marginBottom': '20px', 'fontSize':'1rem', 'textAlign':'left'}),
            html.H3('¿Aumentará la temperatura los próximos meses?', style={'color': '#de3100', 'marginBottom': '20px', 'fontSize':'1rem','fontWeight':'900', 'textAlign':'left'}),
            html.Ul(style={'display': 'flex', 'justifyContent': 'center','flexDirection':'column', 'alignItems':'center', 'width':'100%','margin':'10px 0','padding':'0'},children =[
                dcc.Link('Inicio', href='/', style={'color': 'white', 'fontSize': '15px','width': '80%','marginBottom':'8px', 'textDecoration': 'none', 'padding': '10px', 'borderRadius': '5px', 'backgroundColor': '#ffa888', 'textAlign': 'center'}),
                dcc.Link('Serie de tiempo (EDA)', href='/time-series', style={'color': 'white','width': '80%','marginBottom':'8px',  'fontSize': '15px', 'textDecoration': 'none', 'padding': '10px', 'borderRadius': '5px', 'backgroundColor': '#ffa888', 'textAlign': 'center'}),
                dcc.Link('Modelo Predictivo', href='/predictive-model', style={'color': 'white','width': '80%','marginBottom':'8px',  'fontSize': '15px', 'textDecoration': 'none', 'padding': '10px', 'borderRadius': '5px', 'backgroundColor': '#ffa888', 'textAlign': 'center'}),

            ]),
            html.Div(id='explain-content')
        ]),

        html.Section(id='page-content', className='section',style={'gridArea':'section', 'gridColumnStart':'2','gridColumnEnd':'4','gridRowStart':'1','gridRowEnd':'5','backgroundColor': 'white','border':'1px solid #cbc4ad', 'borderRadius':'5px', 'padding': '20px'}),
    ]),
])

@app.callback(Output('explain-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/time-series':
        return html.Div() 
    elif pathname == '/predictive-model':
        return ()
    else:
        return (
            )
      
# Update the index
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/time-series':
        return (html.Div([
                 html.H2('Estás en la página del análisis exploratorio de la serie de tiempo')
                ])
            )
    elif pathname == '/predictive-model':
        return html.Div([
            html.H3('Estás en la página del modelo predictivo')
        ])
    else:
        return (
            html.H4('Sobre el conjunto de datos',style={'fontSize':'1rem', 'textAlign':'left'}),
            html.P('Presentamos el análisis de un conjunto de datos llamado HadCRUT5, que mide los cambios de temperatura cercana a la superficie en todo el mundo, entre 1850 a 2018. HadCRUT5 presenta anomalías mensuales promedio de temperatura cercana a la superficie, en relación con el período 1961–1990.', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
            html.P('En general, este conjunto de datos muestra un aumento del calentamiento promedio global desde mediados del siglo 19 y en los últimos años, en consonancia con otros análisis. Este fenómeno se debe a múltiples factores, incluidos una mejor representación del calentamiento del Ártico y una mejor comprensión de los sesgos en evolución en las mediciones sobre la superficie del mar y la tierra.', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
            html.P('Para revisar a detalle el proceso de obtención y procesamiento de los datos, visite la información de referencia. ', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
            html.P('Morice, C. P., Kennedy, J. J., Rayner, N. A., Winn, J. P., Hogan, E., Killick, R. E., Dunn, R. J. H., Osborn, T. J., Jones, P. D., & Simpson, I. R. (2021). An Updated Assessment of Near-Surface Temperature Change From 1850: The HadCRUT5 Data Set. Journal of Geophysical Research: Atmospheres, 126(3), e2019JD032361. https://doi.org/10.1029/2019JD032361', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
        )

if __name__ == '__main__':
    app.run_server(debug=True) # <- For testing purposes
    #app.run_server(debug=True, host='0.0.0.0', port=9000) # <- To Dockerize the Dash
    
```

* Este código, luego de importar bibliotecas y cargar los datos, crea una instancia de la aplicación dash. Dentro de esa instancia, define el diseño de la aplicación utilizando componentes HTML y Dash. Los Callbacks que se ejecutan en respuesta a cambio de los inputs, actualiza el contenido de la sección principal de la página en función del 'url' actual. Se crea el contenido de la página de inicio correspondiente a un texto describiendo el conjunto de datos. Asimismo, se realizan ajustes de forma y presentación de títulos, colores, localización, etc.

* Una vez configurada la plantilla, se procede a desarrollar la sección de análisis exploratorio. A continuación, se decriben cada uno de los componentes. 

## Media móvil de 12 meses

* El código inicia importando las bibliotecas necesarias y cargando un conjunto de datos desde un repositorio en github. Luego, define una función para calcular la media móvil y el intervalo de confianza de los datos. La aplicación Dash se crea y se diseña con un título y el gráfico de la serie de tiempo. Una función callback se define para actualizar el gráfico cuando el usuario coloca el cursor sobre él con la función 'hover', mostrando la media móvil, el intervalo de confianza y la media del año seleccionado. Finalmente, la aplicación se ejecuta.

```Python
# Convertir la columna 'date' a un objeto de fecha y hora
df_timeseries['date'] = pd.to_datetime(df_timeseries['date'])

# Ordenar el DataFrame por la columna 'date' (si aún no está ordenado)
df_timeseries = df_timeseries.sort_values('date')

```

```Python
import dash
from dash import  dcc, html, Input, Output
import datetime as dt
import pandas as pd


timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)


# Definimos la función encargada de calcular la media móvil y el intervalo de 
## En este código '1.96' es el valor z para un intervalo de confianza del 95%.

# Función para calcular la media móvil y el intervalo de confianza
def calculate_rolling_mean(df, window_size=12, num_of_std=1.96):
    df['rolling_mean'] = df['value'].rolling(window=window_size).mean()
    df['rolling_std'] = df['value'].rolling(window=window_size).std()
    df['upper_band'] = df['rolling_mean'] + (df['rolling_std'] * num_of_std)
    df['lower_band'] = df['rolling_mean'] - (df['rolling_std'] * num_of_std)
    return df

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Diseño de la aplicación
app.layout = html.Div([
    html.H2('Serie de tiempo - Anomalía de Temperatura Global (1850 - 2023)',
                style={'display': 'inline',
                       'float': 'left',
                       'font-size': '2.65em',
                       'margin-left': '7px',
                       'font-weight': 'bolder',
                       'font-family': 'Product Sans',
                       'color': "rgba(117, 117, 117, 0.95)",
                       'margin-top': '20px',
                       'margin-bottom': '0'
                       }),
    dcc.Graph(id='timeseries-graph'),
])

# Definir la función callback
@app.callback(Output('timeseries-graph', 'figure'),
              [Input('timeseries-graph', 'hoverData')])
def update_graph(hoverData):
    # Calcular la media móvil y el intervalo de confianza
    df_timeseries = calculate_rolling_mean(df_tseries.copy())  # Asegúrate de tener una copia del DataFrame original
    trace_data = []

    # Agregar la serie temporal original
    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['value'],
        'type': 'line',
        'mode': 'lines',
        'name': 'Original Data',
        'line': {'color': 'blue'}
    })

    # Agregar la media móvil
    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['rolling_mean'],
        'type': 'line',
        'mode': 'lines',
        'name': '12-Month Rolling Mean',
        'line': {'color': 'red'}
    })

    # Agregar el intervalo de confianza
    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['upper_band'],
        'type': 'line',
        'mode': 'lines',
        'name': 'Upper Band',
        'line': {'color': 'green', 'dash': 'dash'}
    })

    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['lower_band'],
        'type': 'line',
        'mode': 'lines',
        'name': 'Lower Band',
        'line': {'color': 'green', 'dash': 'dash'}
    })

    # Marcar la media correspondiente al año cuando se pasa el ratón sobre la gráfica
    if hoverData:
        year_hovered = hoverData['points'][0]['x'].split('-')[0]
        mean_value = df_timeseries[df_timeseries['year'] == int(year_hovered)]['rolling_mean'].iloc[0]
        trace_data.append({
            'x': [hoverData['points'][0]['x']],
            'y': [mean_value],
            'type': 'scatter',
            'mode': 'markers',
            'name': f'Mean {year_hovered}',
            'marker': {'color': 'black', 'size': 10},
            'text': [f'Mean {year_hovered}']
        })

    figure = {
        'data': trace_data,
        'layout': {
            'title': 'Media Móvil de 12 Meses con Intervalo de Confianza',
            'xaxis': {'title': 'Fecha'},
            'yaxis': {'title': 'Valor'},
            'margin': {'b': 30, 'r': 10, 'l': 60, 't': 50},
            'legend': {'x': 0, 'y': 1}
        }
    }

    return figure

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)
```


* Se puede observar una tendencia de aumento en la serie luego de la etapa pre-industrial, lo que sugiere que la serie tiene un comportamiento no estacionario, con medias variables y en aumento para intervalos de 12 meses.

## Media mensual multianual y distribución de datos: Boxplot

* Se graficó un boxplot Multianual por mes, que se utiliza para visualizar la distribución de los datos de Anomalías de Temperatura a lo largo de los meses del año. Cada caja en el gráfico representa un mes específico y muestra la variabilidad de los datos para ese mes a lo largo de varios años. El gráfico está diseñado de tal manera que cada caja (o ‘boxplot’) muestra la mediana (la línea en el medio de la caja), los cuartiles superior e inferior (los bordes superior e inferior de la caja), y los valores atípicos (los puntos por encima y por debajo de las líneas que se extienden desde la caja). Esto proporciona una visión clara de la distribución de los datos, permitiendo identificar tendencias y anomalías.

```Python
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

# Lectura de datos
timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Definir el diseño de la aplicación
app.layout = html.Div([
    html.H1("Boxplot Multianual por Mes"),

    dcc.Graph(id='boxplot'),

    # Puedes agregar más componentes de diseño según sea necesario
])

# Definir la función para actualizar el gráfico
@app.callback(Output('boxplot', 'figure'),
              [Input('boxplot', 'hoverData')])
def update_boxplot(hoverData):
    # Crear el boxplot con Plotly Graph Objects
    fig = go.Figure()

    # Agregar boxplots para cada mes
    for month in df_tseries['month'].unique():
        data = df_tseries[df_tseries['month'] == month]
        fig.add_trace(go.Box(x=data['month'], y=data['value'], name=month, width=0.5))

    # Agregar la línea de tendencia (media multianual)
    trend_line = df_tseries.groupby('month')['value'].mean().reset_index()
    
    # Ordenar los datos por el orden de los meses
    ordered_months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    trend_line['month'] = pd.Categorical(trend_line['month'], categories=ordered_months, ordered=True)
    trend_line = trend_line.sort_values('month')

    fig.add_trace(go.Scatter(x=trend_line['month'], y=trend_line['value'],
                             mode='lines', line=dict(color='black', dash='dash'),
                             name='Media Multianual'))

    # Ajustar el diseño del gráfico
    fig.update_layout(xaxis={'title': 'Mes'}, yaxis={'title': 'Anomalía de Temperatura'},
                      boxmode='group')  # Para agrupar los boxplots en el eje x

    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)

```

* El análisis de Anomalías de temperatura por meses indica que, por lo menos el 50% de los datos para todos los meses, se mantiene por debajo de las temperaturas preindustriales, con magnitudes que varían entre -0.22°C y -0.09°C, siendo el mes de octubre el que presenta temperaturas más similares a este período de referencia. Se conoce que, por lo menos el 25% superior de los datos presenta aumentos de temperatura para todos los meses. 


## Anomalías mensual-anual: mapa de calor

```Python
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

# Lectura de datos
timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)


# Crear la aplicación Dash
app = dash.Dash(__name__)

# Orden de los meses
month_order = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

# Pivotar los datos para tener años en el eje x, meses en el eje y
heatmap_data = df_tseries.pivot(index='month', columns='year', values='value')

# Configurar el orden de los meses en el eje y
heatmap_data = heatmap_data.reindex(month_order)

# Crear la aplicación Dash
app.layout = html.Div([
    html.H1("Mapa de Calor de Anomalías de Temperatura"),

    dcc.Graph(id='heatmap'),

    # Puedes agregar más componentes de diseño según sea necesario
])

# Definir la función para actualizar el gráfico de mapa de calor
@app.callback(Output('heatmap', 'figure'),
              [Input('heatmap', 'hoverData')])
def update_heatmap(hoverData):
    # Crear el mapa de calor con Plotly Graph Objects
    fig = go.Figure()

    # Añadir el mapa de calor
    fig.add_trace(go.Heatmap(x=heatmap_data.columns, y=heatmap_data.index, z=heatmap_data.values,
                             colorscale='RdBu', colorbar=dict(title='Anomalía de Temperatura'), reversescale=True))

    # Ajustar el diseño del gráfico
    fig.update_layout(xaxis={'title': 'Año'}, yaxis={'title': 'Mes', 'categoryorder': 'array', 'categoryarray': month_order},
                      title='Mapa de Calor de Anomalías de Temperatura')

    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


```

* A partir de este mapa de calor se han identificado tendencias significativas en los datos de temperatura. Tras la Revolución Industrial, que tuvo lugar a finales del siglo XIX, se ha registrado un incremento progresivo en las temperaturas hasta el cierre del siglo XX. Con el inicio del nuevo milenio, se ha observado una aceleración notable en este incremento de temperatura. Estos patrones globales coinciden con fenómenos de gran escala como la proliferación de la industrialización y el auge y desarrollo de las áreas urbanas, que son los principales contribuyentes a la emisión de gases de efecto invernadero (GEI) a gran escala. Estos gases son los causantes primordiales del calentamiento global debido a su producción intensiva.

## Promedio decenal

* A continuación, se crea un gráfico de barras. Se define una función de devolución de llamada que se activa cuando el usuario interactúa con el gráfico de barras. Esta función calcula los promedios decenales y actualiza el gráfico de barras.

``` Python
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Leer el DataFrame
# Lectura de datos
timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)

# Convertir la columna 'date' a tipo datetime si aún no lo está
df_tseries['date'] = pd.to_datetime(df_tseries['date'])

# Agregar una columna 'decade' para agrupar por década
df_tseries['decade'] = (df_tseries['year'] // 10) * 10

# Agregar una columna 'color' basada en la condición de la anomalía
df_tseries['color'] = df_tseries['value'].apply(lambda x: 'red' if x > 0 else 'blue')

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Crear la aplicación Dash
app.layout = html.Div([
    html.H1("Promedios Decenales de Anomalías de Temperatura"),

    dcc.Graph(id='bar-chart'),

    # Puedes agregar más componentes de diseño según sea necesario
])

# Definir la función para actualizar el gráfico de barras
@app.callback(Output('bar-chart', 'figure'),
              [Input('bar-chart', 'hoverData')])
def update_bar_chart(hoverData):
    # Calcular los promedios decenales
    decade_means = df_tseries.groupby('decade').agg({'value': 'mean', 'color': 'first'}).reset_index()

    # Crear el gráfico de barras con Plotly Express
    fig = px.bar(decade_means, x='decade', y='value', color='color',
                 labels={'value': 'Anomalía de Temperatura'}, 
                 color_discrete_map={'red': 'red', 'blue': 'blue'},
                 title='Promedios Decenales de Anomalías de Temperatura')

    # Modificar las etiquetas de los ticks en el eje x
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=decade_means['decade'],
            ticktext=[f"{decade}s" for decade in decade_means['decade']]
        ),
        showlegend=False
    )

    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)

```

* La gráfica es consistente con el mapa de calor anterior, mostrando durante el siglo XX, especialmente entre 1940 y 1980, se empiezan a ver estas tendencias de aumento de la temperatura.

## Este año hasta ahora

* El siguiente gráfico agrega líneas con la tendencia mensual de la temperatura de un año específico que seleccione el ususario. El usuario puede comparar cualquier año de preferencia con los años que poseen mayores aumentos; en esta caso, corresponden a 2016, 2019 y 2020. 

```Python
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from datetime import datetime

# Carga tu dataframe
df_tseries = pd.read_csv('./df_timeseries.csv')

# Asegúrate de que los datos estén en el formato correcto
df_tseries['date'] = pd.to_datetime(df_tseries['date'])
df_tseries['year'] = df_tseries['date'].dt.year
df_tseries['month'] = df_tseries['date'].dt.month

# Encuentra los tres años más cálidos, excluyendo el año actual
current_year = datetime.now().year
top_years = df_tseries[df_tseries['year'] != current_year].groupby('year')['value'].mean().nlargest(3).index

# Encuentra los valores máximos para cada mes
max_values = df_tseries.groupby('month')['value'].max()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='year-picker',
        options=[{'label': i, 'value': i} for i in df_tseries['year'].unique()],
        value=[df_tseries['year'].max()],
        multi=True
    ),
    dcc.Graph(id='temperature-graph')
])

@app.callback(
    Output('temperature-graph', 'figure'),
    [Input('year-picker', 'value')]
)
def update_figure(selected_years):
    traces = []
    markers = ['circle', 'square', 'diamond', 'cross', 'x']

    # Añade las líneas para los años más cálidos
    for i, year in enumerate(top_years):
        df_year = df_tseries[df_tseries['year'] == year]
        traces.append(go.Scatter(
            x=df_year['month'],
            y=df_year['value'],
            mode='lines+markers',
            name=str(year),
            marker=dict(
                symbol=markers[i % len(markers)],
                size=10
            )
        ))

    # Añade las líneas para los años seleccionados
    for i, selected_year in enumerate(selected_years):
        df_selected = df_tseries[df_tseries['year'] == selected_year]
        traces.append(go.Scatter(
            x=df_selected['month'],
            y=df_selected['value'],
            mode='lines+markers',
            name=str(selected_year),
            line={'color': 'black', 'width': 2},
            marker=dict(
                symbol=markers[(i + len(top_years)) % len(markers)],
                size=10
            )
        ))

    # Añade la línea para los valores máximos
    traces.append(go.Scatter(
        x=np.arange(1, 13),
        y=max_values,
        mode='lines',
        name='Máximo histórico',
        line={'color': 'darkgrey', 'width': 2.5}
    ))

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'title': 'Mes', 'tickvals': list(range(1, 13)), 'ticktext': ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']},
            yaxis={'title': 'Anomalía de temperatura (°C)'},
            hovermode='closest'
        )
    }

if __name__ == '__main__':
    app.run_server(debug=True)

```

## Verificación de la estacionariedad de la serie

* Se utiliza la prueba estadística Dickey-Fuller y los plots ACF y PACF para verificar si la serie es estacionaria. Esto es indispensable para construir modelos de series temporales, como los modelos ARIMA, ya que la estacionariedad es un supuesto fundamental. Si las propiedades estadísticas de la serie no cambian a lo largo del tiempo, se pueden obtener predicciones más precisas. 

```Python
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc
from dash.dependencies import Input, Output
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import plotly.graph_objs as go
from statsmodels.tsa.stattools import acf
from plotly.subplots import make_subplots


#____ Cargue de datos______________________________________________________________________________________________________

timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)

#____ Cálculos y funciones_________________________________________________________________________________________________
#---- Estadístico ADF------------------------------------------------------------------------------------------------------

result = adfuller(df_tseries.value)
adf_statistic = result[0]
p_value = result[1]

#---- Conjuntos de entrenamiento y prueba----------------------------------------------------------------------------------

n_temp = len(df_tseries.value); n_test = 21 
train_size = n_temp - n_test

train = df_tseries.value.iloc[:train_size]
dates_train = df_tseries.date.iloc[:train_size]
test_21month = df_tseries.value.iloc[train_size:train_size + n_test] 
dates_21month = df_tseries.date.iloc[train_size:train_size + n_test] 


# ____Gráficas________________________________________________________________________________________________________________

#----- Autocorrelación--------------------------------------------------------------------------------------------------------

# Crear una figura con subplots
fig = make_subplots(rows=3, cols=2)

# Agregar los gráficos a los subplots correspondientes
fig.add_trace(go.Scatter(y=df_tseries.value, mode='lines', line=dict(color='blue'), showlegend=False, hovertemplate='Value: %{y}'), row=1, col=1)
fig.add_trace(go.Scatter(y=df_tseries.value.diff(), mode='lines', line=dict(color='red'), showlegend=False, hovertemplate='Diff: %{y}'), row=2, col=1)
fig.add_trace(go.Scatter(y=df_tseries.value.diff().diff(), mode='lines', line=dict(color='green'), showlegend=False, hovertemplate='Diff Diff: %{y}'), row=3, col=1)

# Para agregar los gráficos de autocorrelación, se necesita calcular los valores de autocorrelación
acf_values = acf(df_tseries.value, nlags=2080)
fig.add_trace(go.Scatter(y=acf_values, mode='lines', fill='tozeroy', line=dict(color='blue'), showlegend=False, hovertemplate='ACF: %{y}'), row=1, col=2)

diff1_values = df_tseries.value.diff()
acf_diff1_values = acf(diff1_values.dropna(), nlags=2080)
fig.add_trace(go.Scatter(y=acf_diff1_values, mode='lines', fill='tozeroy', line=dict(color='red'), showlegend=False, hovertemplate='ACF Diff: %{y}'), row=2, col=2)

diff2_values = df_tseries.value.diff().diff()
acf_diff2_values = acf(diff2_values.dropna(), nlags=2080)
fig.add_trace(go.Scatter(y=acf_diff2_values, mode='lines', fill='tozeroy', line=dict(color='green'), showlegend=False, hovertemplate='ACF Diff Diff: %{y}'), row=3, col=2)


# Agregar títulos a las gráficas

fig.add_annotation(dict(text='Serie original', x=0.20, y=1.1, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
fig.add_annotation(dict(text='1er Orden de diferenciación', x=0.15, y=0.66, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
fig.add_annotation(dict(text='2do Orden de diferenciación', x=0.15, y=0.25, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))

fig.add_annotation(dict(text='ACF de la Serie original', x=0.85, y=1.1, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
fig.add_annotation(dict(text='ACF del 1er Orden de diferenciación', x=0.88, y=0.66, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
fig.add_annotation(dict(text='ACF del 2do Orden de diferenciación', x=0.88, y=0.25, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))


#____ Configuración de la aplicación__________________________________________________________________________________________
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='mi-dropdown',
                options=[
                    {'label': 'ARIMA', 'value': '1'},
                    {'label': 'Suavización exponencial', 'value': '2'},
                    {'label': 'Regresión polinomial', 'value': '3'}
                ],
                value='1',
                style={'width': '70%', 'marginLeft': '15%'}
            )
        ], width=6, className='my-4')
    ]),
    html.H1("Serie de tiempo: Anomalía de temperatura (°C)", style={'textAlign': 'center'}),
    html.Div(id='output-container')
])

@app.callback(
    Output('output-container', 'children'),
    [Input('mi-dropdown', 'value')]
)
def update_output(value):
    if value == '1':
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H3("ADF test", style={'font-family': 'Arial'}),
                            html.P(f"Estadística ADF: {format(adf_statistic, '.3f')} > -3.4", style={'font-family': 'Arial'}),
                            html.P(f"Valor p: {format(p_value, '.3f')} > 0.05", style={'font-family': 'Arial'}),
                            html.P('Se acepta de hipótesis nula', style={'font-family': 'Arial'})
                        ])
                    ], color='light', className='mx-auto my-4', style={'width': '80%'})
                ], className='my-4'),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H3("Estacionariedad", style={'font-family': 'Arial'}),
                            html.P("La serie de tiempo original no es estacionaria", style={'font-family': 'Arial'})
                        ])
                    ], color='light', className='mx-auto my-4', style={'width': '80%'})
                ], className='my-4')
            ]),
                html.H2("Diferenciación y Autocorrelación", style={'textAlign': 'center'}),
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=fig)
            ])
        ]),


    ])
    else:
        return html.Div() 

if __name__ == "__main__":
    app.run_server(debug=True)

```

* Las tendencias en la serie muestran que la series no es estacionaria. Por lo tanto, debe aplicarse diferenciación para convertir la serie en estacionaria.

# Modelos predictivos
## Modelo ARIMA

* Iniciamos la construcción del modelo ARIMA con la división de los datos en conjuntos de entrenamiento y prueba. Después, utilizando un ciclo for busca el mejor modelo ARIMA para los datos de entrenamiento, probando diferentes combinaciones de parámetros. Una vez que encuentra el mejor modelo, lo ajusta a los datos de entrenamiento y realiza predicciones. La aplicación Dash representan muestra varios gráficos con los datos de entrenamiento, las predicciones del modelo y los límites de confianza de las predicciones.

```Python
import dash
from dash import html, dcc
from dash import dash_table
import pandas as pd
import plotly.graph_objs as go
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")
import pickle
import glob
from time import process_time
from sklearn.metrics import r2_score
import plotly.express as px

# ____ Cargue de datos___________________________________________________________________________________________________________

timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)

# ____ Cálculos y funciones________________________________________________________________________________________________________

# ---- Definimos el conjunto de prueba y de entrenamiento--------------------------------------------------------------------------
n_temp = len(df_tseries.value)
n_test = 21
train_size = n_temp - n_test

train = df_tseries.value.iloc[:train_size]
dates_train = df_tseries.date.iloc[:train_size]
test_21month = df_tseries.value.iloc[train_size:train_size + n_test]
dates_21month = df_tseries.date.iloc[train_size:train_size + n_test]

train_df = df_tseries[["date", "value"]].iloc[:train_size]
test_21month_df = df_tseries[["date", "value"]].iloc[train_size:train_size + n_test]

# ---- Función para seleccionar el mejor modelo------------------------------------------------------------------------------------

best_order = None
grid = None

if __name__ == '__main__':
    if (len(glob.glob("grid_ARIMA.pkl")) != 0):
        grid, best_order = pickle.load(open('grid_ARIMA.pkl','rb'))
        
    else:
        time_start = process_time()

        best_aic = np.inf
        best_bic = np.inf
        best_mdl = None

        pq_rng = range(3)
        d_rng = range(2)

        for i in pq_rng:
            for d in d_rng:
                for j in pq_rng:
                    try:
                        tmp_mdl = ARIMA(train, order=(i, d, j)).fit()
                        tmp_aic = tmp_mdl.aic
                        if tmp_aic < best_aic:
                            best_aic = tmp_aic
                            best_order = (i, d, j)
                            best_mdl = tmp_mdl
                    except:
                        continue
        grid = ARIMA(train, order=best_order)
        grid = grid.fit()
        time_stop = process_time()

        str_cpu_time = "ARIMA CPU time: " + str((time_stop - time_start) * 0.6) + " minutes"
        # print(str_cpu_time)
        with open('cpu_time.txt', 'w', encoding='utf-8') as f:
            f.write(str_cpu_time)
        pickle.dump((grid, best_order), open('grid_ARIMA.pkl', 'wb'))

    # # ---- Predicciones del modelo-------------------------------------------------------------------------------------------------------
    # predictions = grid.get_prediction(start=1, end=len(train), dynamic=False)
    # pred_conf = predictions.conf_int()

    # # Extraer los valores predichos
    # predicted_values = predictions.predicted_mean

    # # Convertir pred_conf a un array de NumPy
    # pred_conf_np = pred_conf.values

    summary_df = pd.read_html(grid.summary().tables[0].as_html(), header=0, index_col=0)[0]

    # ---- Pronósticos ARIMA Rolling-----------------------------------------------------------------------------------------------------

    def arima_rolling(history, test, best_order):
        predictions = list()
        for t in range(len(test)):
            model = ARIMA(history, order=best_order)
            model_fit = model.fit()
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            obs = test[t]
            history.append(obs)
            #print('predicted=%f, expected=%f' % (yhat, obs))
        return predictions

    # Calcular los pronósticos con arima_rolling
    test_21 = test_21month.tolist()
    yhat_21 = arima_rolling(train.tolist(), test_21, best_order)

    # ---- Cálculo del error de predicción -----------------------------------------------------------------------------------
    def forecast_accuracy(forecast, actual, str_name):
        mape = np.mean(np.abs(forecast - actual) / np.abs(actual))  # MAPE
        mae = np.mean(np.abs(forecast - actual))  # MAE
        rmse = np.mean((forecast - actual) ** 2) ** 0.5  # RMSE
        mse = np.mean((forecast - actual) ** 2)  # MSE
        r2 = r2_score(forecast, actual)

        df_acc = pd.DataFrame({'MAE': [mae],
                               'MSE': [mse],
                               'MAPE': [mape],
                               'RMSE': [rmse],
                               'R2': [r2]},
                              index=[str_name])

        return df_acc

    accuracy_df = forecast_accuracy(np.array(yhat_21), np.array(test_21), "21 meses")

    # ----Figuras pronóstico ARIMA -------------------------------------------------------------------------------------------------------

    fig = px.line()
    fig.add_scatter(x=dates_train[-120:], y=train[-120:], mode='lines', name='Entrenamiento', line=dict(color='green'))
    fig.add_scatter(x=dates_21month, y=test_21, mode='lines', name='Prueba', line=dict(color='blue'))
    fig.add_scatter(x=dates_21month, y=yhat_21, mode='lines', name='Pronóstico', line=dict(color='red'))

    # Configuración adicional del diseño
    fig.update_layout(
        xaxis=dict(tickmode='linear', dtick=int(4)),  # Ajuste del eje x
        xaxis_title='Fecha',  # Título del eje x
        yaxis_title='Anomalía de temperatura (°C)',  # Título del eje y
        title='Gráfico de entrenamiento, prueba y pronóstico',  # Título general
        showlegend=True,  # Mostrar leyenda
        legend=dict(x=0, y=1),  # Posición de la leyenda
    )

    # ____ Configuramos la aplicación__________________________________________________________________________________________________

    app = dash.Dash(__name__)

    app.layout = html.Div([
        dcc.Graph(figure=fig),
        html.H2('Resumen del modelo ARIMA'),
        dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in summary_df.columns],
            data=summary_df.to_dict('records'),
            style_table={'width': '80%', 'margin': 'auto'},
            page_size=len(summary_df),
        ),
        html.H2('Precisión de los Pronósticos'),
        dash_table.DataTable(
            id='accuracy-table',
            columns=[{"name": i, "id": i} for i in accuracy_df.columns],
            data=accuracy_df.to_dict('records'),
            style_table={'width': '80%', 'margin': 'auto'},
            page_size=len(accuracy_df),
        )
    ])

    if __name__ == '__main__':
        app.run_server(debug=True)

```

# Suavización exponencial


```Python

import dash
from dash import html, dcc
import pandas as pd
import plotly.graph_objs as go
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

# Cargue de datos
timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)

# Definimos el conjunto de prueba y de entrenamiento
n_temp = len(df_tseries.value); n_test = 21 
train_size = n_temp - n_test

train = df_tseries.value.iloc[:train_size]
dates_train = df_tseries.date.iloc[:train_size]
test_21month = df_tseries.value.iloc[train_size:train_size + n_test] 
dates_21month = df_tseries.date.iloc[train_size:train_size + n_test] 

# Ajustar el modelo de suavización exponencial y obtener el pronóstico
model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12)
model_fit = model.fit()
predicted_values = model_fit.fittedvalues
pred_conf = model_fit.get_prediction(start=1, end=len(train)).conf_int()

# Configuramos la aplicación
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(
        id='graph-smoothing',
        figure={
            'data': [
                go.Scatter(
                    x=dates_train[1:],
                    y=train.loc[1:],
                    mode='lines',
                    name='Datos de entrenamiento'
                ),
                go.Scatter(
                    x=dates_train[1:],
                    y=predicted_values,  # Usar los valores predichos
                    mode='lines',
                    name='Predicciones'
                ),
               go.Scatter(
                    x=dates_train[1:],
                    y=pred_conf[:, 0],
                    mode='lines',
                    name='Límite inferior del intervalo de confianza',
                    line=dict(color='rgb(255,165,0)')
                ),
                go.Scatter(
                    x=dates_train[1:],
                    y=pred_conf[:, 1],
                    mode='lines',
                    fill='tonexty',
                    name='Límite superior del intervalo de confianza',
                    line=dict(color='rgb(255,165,0)')
                ) 
            ],
            'layout': go.Layout(
                xaxis={'title': 'Año'},
                yaxis={'title': 'Anomalía de temperatura (°C)'},
                autosize=False,
            )
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


```

# Regresión polinómica

```Python
import dash
from dash import html, dcc
import pandas as pd
import plotly.graph_objs as go
from numpy.polynomial.polynomial import Polynomial
import warnings
warnings.filterwarnings("ignore")

# Cargue de datos
timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)

# Definimos el conjunto de prueba y de entrenamiento
n_temp = len(df_tseries.value); n_test = 21 
train_size = n_temp - n_test

train = df_tseries.value.iloc[:train_size]
dates_train = df_tseries.date.iloc[:train_size]
test_21month = df_tseries.value.iloc[train_size:train_size + n_test] 
dates_21month = df_tseries.date.iloc[train_size:train_size + n_test] 

# Ajustar el modelo de regresión polinómica y obtener el pronóstico
x = list(range(len(train)))
y = train.values
degree = 3  # puedes cambiar el grado del polinomio aquí
model = Polynomial.fit(x, y, degree)
predicted_values = model(x)

# Configuramos la aplicación
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(
        id='graph-smoothing',
        figure={
            'data': [
                go.Scatter(
                    x=dates_train,
                    y=train,
                    mode='lines',
                    name='Datos de entrenamiento'
                ),
                go.Scatter(
                    x=dates_train,
                    y=predicted_values,  # Usar los valores predichos
                    mode='lines',
                    name='Predicciones'
                )
            ],
            'layout': go.Layout(
                xaxis={'title': 'Año'},
                yaxis={'title': 'Anomalía de temperatura (°C)'},
                autosize=False,
            )
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

```

# Código unificado

```Python
import dash
from dash import dcc
from dash import html
from dash import dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf
from plotly.subplots import make_subplots
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
import pickle
import glob
from time import process_time
from sklearn.metrics import r2_score
import plotly.express as px


#____ Cargue de datos___________________________________________________________________________________________________________

timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)

#____ Cálculos y funciones______________________________________________________________________________________________________

#---- Estadístico ADF-----------------------------------------------------------------------------------------------------------

result = adfuller(df_tseries.value)
adf_statistic = result[0]
p_value = result[1]

#---- Conjuntos de entrenamiento y prueba----------------------------------------------------------------------------------------

n_temp = len(df_tseries.value); n_test = 21 
train_size = n_temp - n_test

train = df_tseries.value.iloc[:train_size]
dates_train = df_tseries.date.iloc[:train_size]
test_21month = df_tseries.value.iloc[train_size:train_size + n_test] 
dates_21month = df_tseries.date.iloc[train_size:train_size + n_test] 

train_df = df_tseries[["date", "value"]].iloc[:train_size]
test_21month_df = df_tseries[["date", "value"]].iloc[train_size:train_size + n_test] 


#----Función para seleccionar el mejor modelo------------------------------------------------------------------------------------
best_aic = np.inf
best_bic = np.inf

best_order = None
best_mdl = None

pq_rng = range(5)
d_rng  = range(3)

for i in pq_rng:
    for d in d_rng:
        for j in pq_rng:
            try:
                # print(i, d, j)
                tmp_mdl = ARIMA(train, order=(i,d,j)).fit()
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic:
                    best_aic = tmp_aic
                    best_order = (i, d, j)
                    best_mdl = tmp_mdl
            except: continue

#----Ajustar el modelo ARIMA y obtener el pronóstico------------------------------------------------------------------------------

model = ARIMA(train, order=best_order)
model_fit = model.fit()
predictions = model_fit.get_prediction(start=1, end=len(train), dynamic=False)
pred_conf = predictions.conf_int()

# Extraer los valores predichos
predicted_values = predictions.predicted_mean

# Convertir pred_conf a un array de NumPy
pred_conf_np = pred_conf.values

# ____Gráficas___________________________________________________________________________________________________________________
####ARIMA MODEL PREDICTIVE
###########################
###########################

grid = None

if __name__ == '__main__':
    if (len(glob.glob("grid_ARIMA.pkl")) != 0):
        grid, best_order = pickle.load(open('grid_ARIMA.pkl','rb'))
        
    else:
        time_start = process_time()

        best_aic = np.inf
        best_bic = np.inf
        best_mdl = None

        pq_rng = range(3)
        d_rng = range(2)

        for i in pq_rng:
            for d in d_rng:
                for j in pq_rng:
                    try:
                        tmp_mdl = ARIMA(train, order=(i, d, j)).fit()
                        tmp_aic = tmp_mdl.aic
                        if tmp_aic < best_aic:
                            best_aic = tmp_aic
                            best_order = (i, d, j)
                            best_mdl = tmp_mdl
                    except:
                        continue
        grid = ARIMA(train, order=best_order)
        grid = grid.fit()
        time_stop = process_time()

        str_cpu_time = "ARIMA CPU time: " + str((time_stop - time_start) * 0.6) + " minutes"
        # print(str_cpu_time)
        with open('cpu_time.txt', 'w', encoding='utf-8') as f:
            f.write(str_cpu_time)
        pickle.dump((grid, best_order), open('grid_ARIMA.pkl', 'wb'))

    # # ---- Predicciones del modelo-------------------------------------------------------------------------------------------------------
    # predictions = grid.get_prediction(start=1, end=len(train), dynamic=False)
    # pred_conf = predictions.conf_int()

    # # Extraer los valores predichos
    # predicted_values = predictions.predicted_mean

    # # Convertir pred_conf a un array de NumPy
    # pred_conf_np = pred_conf.values

    summary_df = pd.read_html(grid.summary().tables[0].as_html(), header=0, index_col=0)[0]

    # ---- Pronósticos ARIMA Rolling-----------------------------------------------------------------------------------------------------

    def arima_rolling(history, test, best_order):
        predictions = list()
        for t in range(len(test)):
            model = ARIMA(history, order=best_order)
            model_fit = model.fit()
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            obs = test[t]
            history.append(obs)
            #print('predicted=%f, expected=%f' % (yhat, obs))
        return predictions

    # Calcular los pronósticos con arima_rolling
    test_21 = test_21month.tolist()
    yhat_21 = arima_rolling(train.tolist(), test_21, best_order)

    # ---- Cálculo del error de predicción -----------------------------------------------------------------------------------
    def forecast_accuracy(forecast, actual, str_name):
        mape = np.mean(np.abs(forecast - actual) / np.abs(actual))  # MAPE
        mae = np.mean(np.abs(forecast - actual))  # MAE
        rmse = np.mean((forecast - actual) ** 2) ** 0.5  # RMSE
        mse = np.mean((forecast - actual) ** 2)  # MSE
        r2 = r2_score(forecast, actual)

        df_acc = pd.DataFrame({'MAE': [mae],
                               'MSE': [mse],
                               'MAPE': [mape],
                               'RMSE': [rmse],
                               'R2': [r2]},
                              index=[str_name])

        return df_acc

    accuracy_df = forecast_accuracy(np.array(yhat_21), np.array(test_21), "21 meses")

    # ----Figuras pronóstico ARIMA -------------------------------------------------------------------------------------------------------

    fig1 = px.line()
    fig1.add_scatter(x=dates_train[-120:], y=train[-120:], mode='lines', name='Entrenamiento', line=dict(color='green'))
    fig1.add_scatter(x=dates_21month, y=test_21, mode='lines', name='Prueba', line=dict(color='blue'))
    fig1.add_scatter(x=dates_21month, y=yhat_21, mode='lines', name='Pronóstico', line=dict(color='red'))

    # Configuración adicional del diseño
    fig1.update_layout(
        xaxis=dict(tickmode='linear', dtick=int(4)),  # Ajuste del eje x
        xaxis_title='Fecha',  # Título del eje x
        yaxis_title='Anomalía de temperatura (°C)',  # Título del eje y
        title='Gráfico de entrenamiento, prueba y pronóstico',  # Título general
        showlegend=True,  # Mostrar leyenda
        legend=dict(x=0, y=1),  # Posición de la leyenda
    )

####### END ARIMA MODEL PREDICTIVE#########




#----- Autocorrelación-----------------------------------------------------------------------------------------------------------

# Crear una figura con subplots
fig = make_subplots(rows=3, cols=2)

# Agregar los gráficos a los subplots correspondientes
fig.add_trace(go.Scatter(y=df_tseries.value, mode='lines', line=dict(color='blue'), showlegend=False, hovertemplate='Value: %{y}'), row=1, col=1)
fig.add_trace(go.Scatter(y=df_tseries.value.diff(), mode='lines', line=dict(color='red'), showlegend=False, hovertemplate='Diff: %{y}'), row=2, col=1)
fig.add_trace(go.Scatter(y=df_tseries.value.diff().diff(), mode='lines', line=dict(color='green'), showlegend=False, hovertemplate='Diff Diff: %{y}'), row=3, col=1)

# Para agregar los gráficos de autocorrelación, se necesita calcular los valores de autocorrelación
acf_values = acf(df_tseries.value, nlags=2080)
fig.add_trace(go.Scatter(y=acf_values, mode='lines', fill='tozeroy', line=dict(color='blue'), showlegend=False, hovertemplate='ACF: %{y}'), row=1, col=2)

diff1_values = df_tseries.value.diff()
acf_diff1_values = acf(diff1_values.dropna(), nlags=2080)
fig.add_trace(go.Scatter(y=acf_diff1_values, mode='lines', fill='tozeroy', line=dict(color='red'), showlegend=False, hovertemplate='ACF Diff: %{y}'), row=2, col=2)

diff2_values = df_tseries.value.diff().diff()
acf_diff2_values = acf(diff2_values.dropna(), nlags=2080)
fig.add_trace(go.Scatter(y=acf_diff2_values, mode='lines', fill='tozeroy', line=dict(color='green'), showlegend=False, hovertemplate='ACF Diff Diff: %{y}'), row=3, col=2)


# Agregar títulos a las gráficas

fig.add_annotation(dict(text='Serie original', x=0.20, y=1.1, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
fig.add_annotation(dict(text='1er Orden de diferenciación', x=0.15, y=0.66, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
fig.add_annotation(dict(text='2do Orden de diferenciación', x=0.15, y=0.25, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))

fig.add_annotation(dict(text='ACF de la Serie original', x=0.85, y=1.1, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
fig.add_annotation(dict(text='ACF del 1er Orden de diferenciación', x=0.88, y=0.66, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
fig.add_annotation(dict(text='ACF del 2do Orden de diferenciación', x=0.88, y=0.25, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))


#-----Media móvil-----------------------------------------------------------------------------------------------------------

## En este código '1.96' es el valor z para un intervalo de confianza del 95%.

# Función para calcular la media móvil y el intervalo de confianza
def calculate_rolling_mean(df, window_size=12, num_of_std=1.96):
    df['rolling_mean'] = df['value'].rolling(window=window_size).mean()
    df['rolling_std'] = df['value'].rolling(window=window_size).std()
    df['upper_band'] = df['rolling_mean'] + (df['rolling_std'] * num_of_std)
    df['lower_band'] = df['rolling_mean'] - (df['rolling_std'] * num_of_std)
    return df

# Orden de los meses
month_order = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
# Pivotar los datos para tener años en el eje x, meses en el eje y
heatmap_data = df_tseries.pivot(index='month', columns='year', values='value')
# Configurar el orden de los meses en el eje y
heatmap_data = heatmap_data.reindex(month_order)


# Convertir la columna 'date' a tipo datetime si aún no lo está
df_tseries['date'] = pd.to_datetime(df_tseries['date'])
# Agregar una columna 'decade' para agrupar por década
df_tseries['decade'] = (df_tseries['year'] // 10) * 10
# Agregar una columna 'color' basada en la condición de la anomalía
df_tseries['color'] = df_tseries['value'].apply(lambda x: 'red' if x > 0 else 'blue')

df_tseries['year'] = df_tseries['date'].dt.year
df_tseries['month'] = df_tseries['date'].dt.month
# Encuentra los tres años más cálidos, excluyendo el año actual
current_year = datetime.now().year
top_years = df_tseries[df_tseries['year'] != current_year].groupby('year')['value'].mean().nlargest(3).index
# Encuentra los valores máximos para cada mes
max_values = df_tseries.groupby('month')['value'].max()


#____Configuración de la aplicación_____________________________________________________________________________________________

app = dash.Dash(__name__, suppress_callback_exceptions=True)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Main(style={'display':'grid', 'height':'100%','minHeight':'100vh','gridTemplateColumns': '50vh 1fr', 'gridTemplateRows': '50px 1fr 30px', }, children=[
       
        html.Nav(className='nav', style={'display':'fixed','gridArea':'nav', 'gridColumnStart':'1','gridColumnEnd':'2','gridRowStart':'1','gridRowEnd':'5', 'backgroundColor': 'white','border':'1px solid #cbc4ad', 'borderRadius':'5px', 'padding': '20px'}, children=[
            html.H1('Anomalia de Temperatura de Superficie Global', style={'color': 'black', 'marginBottom': '20px', 'fontSize':'1.5rem', 'textAlign':'center'}),
            html.H2('¿Cuáles han sido las variaciones más significativas de la temperatura de la superficie global a lo largo del tiempo?', style={'color': 'black', 'marginBottom': '20px', 'fontSize':'1rem', 'textAlign':'left'}),
            html.H3('¿Aumentará la temperatura los próximos meses?', style={'color': '#de3100', 'marginBottom': '20px', 'fontSize':'1rem','fontWeight':'900', 'textAlign':'left'}),
            html.Ul(style={'display': 'flex', 'justifyContent': 'center','flexDirection':'column', 'alignItems':'center', 'width':'100%','margin':'10px 0','padding':'0'},children =[
                dcc.Link('Inicio', href='/', style={'color': 'white', 'fontSize': '15px','width': '80%','marginBottom':'8px', 'textDecoration': 'none', 'padding': '10px', 'borderRadius': '5px', 'backgroundColor': '#ffa888', 'textAlign': 'center'}),
                dcc.Link('Serie de tiempo', href='/time-series', style={'color': 'white','width': '80%','marginBottom':'8px',  'fontSize': '15px', 'textDecoration': 'none', 'padding': '10px', 'borderRadius': '5px', 'backgroundColor': '#ffa888', 'textAlign': 'center'}),
                dcc.Link('Modelo Predictivo', href='/predictive-model', style={'color': 'white','width': '80%','marginBottom':'8px',  'fontSize': '15px', 'textDecoration': 'none', 'padding': '10px', 'borderRadius': '5px', 'backgroundColor': '#ffa888', 'textAlign': 'center'}),

            ]),
            html.Div(id='explain-content')
        ]),

        html.Section(id='page-content', className='section',style={'gridArea':'section', 'gridColumnStart':'2','gridColumnEnd':'4','gridRowStart':'1','gridRowEnd':'5','backgroundColor': 'white','border':'1px solid #cbc4ad', 'borderRadius':'5px', 'padding': '20px'}),
    ]),
])

@app.callback(Output('explain-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/time-series':
        return html.Div() 
    elif pathname == '/predictive-model':
        return ()
    else:
        return (
            )
      
# Update the index
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/time-series':
        return (html.Div([
                 html.H2('Serie de tiempo - Anomalía de Temperatura Global (1850 - 2023)'),
                dcc.Graph(id='timeseries-graph'),
                ]),
                html.Div([
                    html.H2(" Boxplot - Variación mensual multianual (1850 - 2023)",),
                    dcc.Graph(id='boxplot')]),
                html.Div([
                    html.Div([
                        html.H2("Heatmap- Exploración de patrones y tendencias"),
                        dcc.Graph(id='heatmap'),]),
                    html.Div([
                        html.H1("Barplot - Promedios Decenales de Anomalías de Temperatura"),
                        dcc.Graph(id='bar-chart'),
                    ])
                ]),
                html.Div([
                    dcc.Dropdown(
                        id='year-picker',
                        options=[{'label': i, 'value': i} for i in df_tseries['year'].unique()],
                        value=[df_tseries['year'].max()],
                        multi=True
                    ),
                     dcc.Graph(id='temperature-graph')
                ]), 
                    html.Div([
                        html.H2("Análisis de estacionariedad",),
                        html.Div(id='output-container', children=[

                        ]),
                ]),
            )
    elif pathname == '/predictive-model':
        return (html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Dropdown(
                            id='mi-dropdown',
                            options=[
                                {'label': 'ARIMA', 'value': '1'},
                                {'label': 'Suavización exponencial', 'value': '2'},
                                {'label': 'Regresión polinomial', 'value': '3'}
                            ],
                            value='1',
                            style={'width': '70%', 'marginLeft': '15%'}
                        )
                    ], width=6, className='my-4')
                ]),
                html.H1("Serie de tiempo: Anomalía de temperatura (°C)", style={'textAlign': 'center'}),
            ]),
            html.Div(id='graph', children=[
               
            ]),
        )
    else:
        return (
            html.H4('Sobre el conjunto de datos',style={'fontSize':'1rem', 'textAlign':'left'}),
            html.P('Presentamos el análisis de un conjunto de datos llamado HadCRUT5, que mide los cambios de temperatura cercana a la superficie en todo el mundo, entre 1850 a 2018. HadCRUT5 presenta anomalías mensuales promedio de temperatura cercana a la superficie, en relación con el período 1961–1990.', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
            html.P('En general, este conjunto de datos muestra un aumento del calentamiento promedio global desde mediados del siglo 19 y en los últimos años, en consonancia con otros análisis. Este fenómeno se debe a múltiples factores, incluidos una mejor representación del calentamiento del Ártico y una mejor comprensión de los sesgos en evolución en las mediciones sobre la superficie del mar y la tierra.', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
            html.P('Para revisar a detalle el proceso de obtención y procesamiento de los datos, visite la información de referencia. ', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
            html.P('Morice, C. P., Kennedy, J. J., Rayner, N. A., Winn, J. P., Hogan, E., Killick, R. E., Dunn, R. J. H., Osborn, T. J., Jones, P. D., & Simpson, I. R. (2021). An Updated Assessment of Near-Surface Temperature Change From 1850: The HadCRUT5 Data Set. Journal of Geophysical Research: Atmospheres, 126(3), e2019JD032361. https://doi.org/10.1029/2019JD032361', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
        )


# Definir serie de tiempo
@app.callback(Output('timeseries-graph', 'figure'),
              [Input('timeseries-graph', 'hoverData')])
def update_graph(hoverData):
    # Calcular la media móvil y el intervalo de confianza
    df_timeseries = calculate_rolling_mean(df_tseries.copy())  # Asegúrate de tener una copia del DataFrame original
    trace_data = []

    # Agregar la serie temporal original
    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['value'],
        'type': 'line',
        'mode': 'lines',
        'name': 'Serie de tiempo',
        'line': {'color': 'blue'}
    })

    # Agregar la media móvil
    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['rolling_mean'],
        'type': 'line',
        'mode': 'lines',
        'name': 'Media móvil de 12 meses',
        'line': {'color': 'red'}
    })

    # Agregar el intervalo de confianza
    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['upper_band'],
        'type': 'line',
        'mode': 'lines',
        'name': 'IC - 95%',
        'line': {'color': 'green', 'dash': 'dash'}
    })

    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['lower_band'],
        'type': 'line',
        'mode': 'lines',
        'line': {'color': 'green', 'dash': 'dash'}
    })

    # Marcar la media correspondiente al año cuando se pasa el ratón sobre la gráfica
    if hoverData:
        year_hovered = hoverData['points'][0]['x'].split('-')[0]
        mean_value = df_timeseries[df_timeseries['year'] == int(year_hovered)]['rolling_mean'].iloc[0]
        trace_data.append({
            'x': [hoverData['points'][0]['x']],
            'y': [mean_value],
            'type': 'scatter',
            'mode': 'markers',
            'name': f'Mean {year_hovered}',
            'marker': {'color': 'black', 'size': 10},
            'text': [f'Mean {year_hovered}']
        })

    figure = {
        'data': trace_data,
        'layout': {
            'xaxis': {'title': 'Fecha'},
            'yaxis': {'title': 'Anomalía de temperatura (°C)'},
            'margin': {'b': 30, 'r': 10, 'l': 60, 't': 50},
            'legend': {'x': 0, 'y': 1}
        }
    }

    return figure

# Callback boxplot
@app.callback(Output('boxplot', 'figure'),
              [Input('boxplot', 'hoverData')])
def update_boxplot(hoverData):
    # Crear el boxplot con Plotly Graph Objects
    fig = go.Figure()

    # Agregar boxplots para cada mes
    for month in df_tseries['month'].unique():
        data = df_tseries[df_tseries['month'] == month]
        fig.add_trace(go.Box(x=data['month'], y=data['value'], name=str(month), width=0.5, showlegend=False))

    # Agregar la línea de tendencia (media multianual)
    trend_line = df_tseries.groupby('month')['value'].mean().reset_index()
    
    # Ordenar los datos por el orden de los meses
    ordered_months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    trend_line['month'] = pd.Categorical(trend_line['month'], categories=ordered_months, ordered=True)
    trend_line = trend_line.sort_values('month')

    fig.add_trace(go.Scatter(x=trend_line['month'], y=trend_line['value'],
                             mode='lines', line=dict(color='black', dash='dash'),
                             name='Media Multianual', showlegend=False))

    # Ajustar el diseño del gráfico
    fig.update_layout(xaxis={'title': 'Mes', 'tickmode': 'array', 'tickvals': list(range(1, 13)), 'ticktext': ordered_months}, yaxis={'title': 'Anomalía de Temperatura (°C)'},
                      boxmode='group')  # Para agrupar los boxplots en el eje x

    return fig

# Callback heatmap
@app.callback(Output('heatmap', 'figure'),
              [Input('heatmap', 'hoverData')])
def update_heatmap(hoverData):
    # Crear el mapa de calor con Plotly Graph Objects
    fig = go.Figure()

    # Añadir el mapa de calor
    fig.add_trace(go.Heatmap(x=heatmap_data.columns, y=heatmap_data.index, z=heatmap_data.values,
                             colorscale='RdBu', colorbar=dict(title='Anomalía de Temperatura'), reversescale=True))

    # Ajustar el diseño del gráfico
    fig.update_layout(xaxis={'title': 'Año'}, yaxis={'title': 'Mes', 'categoryorder': 'array', 'categoryarray': month_order})

    return fig

# Callback bar-chart
@app.callback(Output('bar-chart', 'figure'),
              [Input('bar-chart', 'hoverData')])
def update_bar_chart(hoverData):
    # Calcular los promedios decenales
    decade_means = df_tseries.groupby('decade').agg({'value': 'mean', 'color': 'first'}).reset_index()

    # Crear el gráfico de barras con Plotly Express
    fig = px.bar(decade_means, x='decade', y='value', color='color',
                 labels={'value': 'Anomalía de Temperatura (°C)'}, 
                 color_discrete_map={'red': 'red', 'blue': 'blue'})

    # Modificar las etiquetas de los ticks en el eje x
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=decade_means['decade'],
            ticktext=[f"{decade}s" for decade in decade_means['decade']]
        ),
        showlegend=False
    )

    return fig

# Callback año máximo
@app.callback(
    Output('temperature-graph', 'figure'),
    [Input('year-picker', 'value')]
)
def update_figure(selected_years):
    traces = []
    markers = ['circle', 'square', 'diamond', 'cross', 'x']

    # Añade las líneas para los años más cálidos
    for i, year in enumerate(top_years):
        df_year = df_tseries[df_tseries['year'] == year]
        traces.append(go.Scatter(
            x=df_year['month'],
            y=df_year['value'],
            mode='lines+markers',
            name=str(year),
            marker=dict(
                symbol=markers[i % len(markers)],
                size=10
            )
        ))

    # Añade las líneas para los años seleccionados
    for i, selected_year in enumerate(selected_years):
        df_selected = df_tseries[df_tseries['year'] == selected_year]
        traces.append(go.Scatter(
            x=df_selected['month'],
            y=df_selected['value'],
            mode='lines+markers',
            name=str(selected_year),
            line={'color': 'black', 'width': 2},
            marker=dict(
                symbol=markers[(i + len(top_years)) % len(markers)],
                size=10
            )
        ))

    # Añade la línea para los valores máximos
    traces.append(go.Scatter(
        x=np.arange(1, 13),
        y=max_values,
        mode='lines',
        name='Máximo histórico (1850 - 2022)',
        line={'color': 'darkgrey', 'width': 2.5}
    ))

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'title': 'Mes', 'tickvals': list(range(1, 13)), 'ticktext': ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']},
            yaxis={'title': 'Anomalía de temperatura (°C)'},
            hovermode='closest'
        )
    }

#Callback verificación de estacionariedad
@app.callback(
    Output('output-container', 'children'),
    [Input('output-container', 'value')]
)
def update_output(value):
    
        return (dbc.Container([
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H3("ADF test", style={'font-family': 'Arial'}),
                            html.P(f"Estadística ADF: {format(adf_statistic, '.3f')} > -3.4", style={'font-family': 'Arial'}),
                            html.P(f"Valor p: {format(p_value, '.3f')} > 0.05", style={'font-family': 'Arial'}),
                            html.P('Se acepta de hipótesis nula', style={'font-family': 'Arial'})
                        ])
                    ], color='light', className='mx-auto my-4', style={'width': '80%'})
                ], className='my-4'),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H3("Estacionariedad", style={'font-family': 'Arial'}),
                            html.P("La serie de tiempo original no es estacionaria", style={'font-family': 'Arial'})
                        ])
                    ], color='light', className='mx-auto my-4', style={'width': '80%'})
                ], className='my-4')
            ]),
                html.H2("Diferenciación y Autocorrelación", style={'textAlign': 'center'}),
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=fig)
            ])
        ]),


    ]))

# Callback gráfico datos de entrenamiento
@app.callback(Output('traindata-graph', 'figure'),
              [Input('traindata-graph', 'hoverData')])
def update_graph(hoverData):
    trace_data = []

    # Agregar la serie de entrenamiento
    trace_data.append({
        'x': dates_train,
        'y': train,
        'type': 'line',
        'mode': 'lines',
        'name': 'Entrenamiento',
        'line': {'color': 'purple'}
    })

    # Agregar la serie de prueba
    trace_data.append({
        'x': dates_21month,
        'y': test_21month,
        'type': 'line',
        'mode': 'lines',
        'name': 'Prueba',
        'line': {'color': 'orange'}
    })

    # Marcar la media correspondiente al año cuando se pasa el ratón sobre la gráfica
    if hoverData:
        year_hovered = hoverData['points'][0]['x'].split('-')[0]
        mean_value = df_tseries[df_tseries['year'] == int(year_hovered)]['rolling_mean'].iloc[0]
        trace_data.append({
            'x': [hoverData['points'][0]['x']],
            'y': [mean_value],
            'type': 'scatter',
            'mode': 'markers',
            'name': f'Media {year_hovered}',
            'marker': {'color': 'black', 'size': 10},
            'text': [f'Media {year_hovered}']
        })

    figure = {
        'data': trace_data,
        'layout': {
            'xaxis': {'title': 'Fecha'},
            'yaxis': {'title': 'Anomalía de temperatura (°C)'},
            'margin': {'b': 30, 'r': 10, 'l': 60, 't': 50},
            'legend': {'x': 0, 'y': 1}
        }
    }

    return figure


#CallBack que renderiza dependiendo del id del dropdown
@app.callback(
    Output('graph', 'children'),
    [Input('mi-dropdown', 'value')]
)
def update_graph(selected_value):
    if selected_value == '1':
        # Genera la gráfica para ARIMA
        return(
            html.Div([
                dcc.Graph(figure=fig1),
                html.H2('Resumen del modelo ARIMA'),
                dash_table.DataTable(
                    id='table',
                    columns=[{"name": i, "id": i} for i in summary_df.columns],
                    data=summary_df.to_dict('records'),
                    style_table={'width': '80%', 'margin': 'auto'},
                    page_size=len(summary_df),
                ),
                html.H2('Precisión de los Pronósticos'),
                dash_table.DataTable(
                    id='accuracy-table',
                    columns=[{"name": i, "id": i} for i in accuracy_df.columns],
                    data=accuracy_df.to_dict('records'),
                    style_table={'width': '80%', 'margin': 'auto'},
                    page_size=len(accuracy_df),
                )
            ])
  )  
    elif selected_value == '2':
        # Genera la gráfica para Suavización exponencial
        return (html.Div("Suavización exponencial"))
           
        
    elif selected_value == '3':
        # Genera la gráfica para Regresión polinomial
       return (html.Div("Regresión polinomial"))

    



if __name__ == '__main__':
    app.run_server(debug=True) # <- For testing purposes
    # app.run_server(debug=True, host='0.0.0.0', port=9000) # <- To Dockerize the Dash
    


```